<a href="https://colab.research.google.com/github/GabrielaFuller/Aether/blob/main/CSC580_Final_Project_Enhanced.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Gabriela Fuller
CSC 580
Final Project


In [22]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [23]:
#Import the necessary libraries
import gym
from gym import spaces
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [24]:

# read csv into dataframe
file_path = "/content/test_stock.csv"
df = pd.read_csv(file_path) # Changed 'test_stock' to 'file_path'

df = df[['Date', 'Close']]
df = df.rename(columns={'Close': 'actual', 'Date': 'date'})

# convert dates from object to DateTime type
dates = df['date']
dates = pd.to_datetime(dates, infer_datetime_format=True)
df['date'] = dates

df.head()

<ipython-input-24-cb841bc19c19>:10: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  dates = pd.to_datetime(dates, infer_datetime_format=True)
<ipython-input-24-cb841bc19c19>:10: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  dates = pd.to_datetime(dates, infer_datetime_format=True)


,date,actual
0,2024-08-01 00:00:00-04:00,61.020000
1,2024-08-01 00:00:00-04:00,58.169998
2,2024-08-01 00:00:00-04:00,147.020004
3,2024-08-01 00:00:00-04:00,100.930000
4,2024-08-01 00:00:00-04:00,121.550003


In [25]:
import numpy as np
import random
import tensorflow as tf
from collections import deque

class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95  # discount rate
        self.epsilon = 1.0  # exploration rate
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.model = self._build_model()

    def _build_model(self):
        model = tf.keras.Sequential()
        model.add(tf.keras.layers.Dense(24, input_dim=self.state_size, activation='relu'))
        model.add(tf.keras.layers.Dense(24, activation='relu'))
        model.add(tf.keras.layers.Dense(self.action_size, activation='linear'))
        model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(lr=0.001))
        return model

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)  # Explore
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])  # Exploit

    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = reward
            if not done:
                target += self.gamma * np.max(self.model.predict(next_state))
            target_f = self.model.predict(state)
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1, verbose=0)

    def load(self, name):
        self.model.load_weights(name)

    def save(self, name):
        self.model.save_weights(name)

In [26]:
#create trading environment


class StockTradingEnv(gym.Env):
    def __init__(self, data):
        super(StockTradingEnv, self).__init__()
        self.data = data
        self.current_step = 0
        self.action_space = spaces.Discrete(3)  # Buy, Hold, Sell
        self.observation_space = spaces.Box(low=0, high=np.inf, shape=(len(data.columns),), dtype=np.float32)
        self.current_balance = 1000  # Starting balance
        self.current_stock = 0  # Number of stocks owned

    def reset(self):
        self.current_step = 0
        self.current_balance = 1000
        self.current_stock = 0
        return self.data.iloc[self.current_step].values

    def step(self, action):
        current_price = self.data.iloc[self.current_step]['Close']
        reward = 0

        if action == 0:  # Buy
            if self.current_balance >= current_price:
                self.current_stock += 1
                self.current_balance -= current_price
        elif action == 1:  # Hold
            reward = 0
        elif action == 2:  # Sell
            if self.current_stock > 0:
                self.current_stock -= 1
                self.current_balance += current_price
                reward = current_price - self.data.iloc[self.current_step]['Close']  # Profit from selling

        self.current_step += 1
        done = self.current_step >= len(self.data) - 1
        next_state = self.data.iloc[self.current_step].values
        total_value = self.current_balance + self.current_stock * current_price
        return next_state, reward, done, {'total_value': total_value}

    def render(self):
        total_value = self.current_balance + self.current_stock * self.data.iloc[self.current_step]['Close']
        print(f'Step: {self.current_step}, Total Value: {total_value}, Balance: {self.current_balance}, Stocks: {self.current_stock}')

In [27]:
#deep-q learning model

import tensorflow as tf
from tensorflow.keras import layers

def create_model(state_size, action_size):
    model = tf.keras.Sequential()
    model.add(layers.Dense(24, activation='relu', input_shape=(state_size,)))
    model.add(layers.Dense(24, activation='relu'))
    model.add(layers.Dense(action_size, activation='linear'))
    model.compile(optimizer='adam', loss='mse')
    return model

In [28]:
import random

def train_dqn(env, model, episodes, gamma=0.95, epsilon=1.0, epsilon_decay=0.995, epsilon_min=0.01):
    for e in range(episodes):
        state = env.reset()
        done = False
        total_reward = 0

        while not done:
            if np.random.rand() <= epsilon:
                action = random.randrange(env.action_space.n)  # Explore
            else:
                q_values = model.predict(state.reshape(1, -1))
                action = np.argmax(q_values[0])  # Exploit

            next_state, reward, done, _ = env.step(action)
            total_reward += reward

import random
import numpy as np

def train_dqn(env, model, episodes, gamma=0.95, epsilon=1.0, epsilon_decay=0.995, epsilon_min=0.01):
    for e in range(episodes):
        state = env.reset()
        done = False
        total_reward = 0

        while not done:
            if np.random.rand() <= epsilon:
                action = random.randrange(env.action_space.n)  # Explore
            else:
                q_values = model.predict(state.reshape(1, -1))
                action = np.argmax(q_values[0])  # Exploit

            next_state, reward, done, _ = env.step(action)
            total_reward += reward

            # Update Q-value
            target = reward + gamma * np.max(model.predict(next_state.reshape(1, -1))[0])
            target_f = model.predict(state.reshape(1, -1))
            target_f[0][action] = target
            model.fit(state.reshape(1, -1), target_f, epochs=1, verbose=0)
            state = next_state

            if done:
                print(f"Episode: {e}/{episodes}, Total Reward: {total_reward}")
                break

            if epsilon > epsilon_min:
                epsilon *= epsilon_decay

In [32]:
#training the agent


import gym
import yfinance as yf

# Fetch historical data for a stock
ticker = 'AAPL'
data = yf.download(ticker, start='2010-01-01', end='2023-01-01')
data = data[['Open', 'High', 'Low', 'Close', 'Volume']]
data.reset_index(inplace=True)

# Create an instance of the StockTradingEnv
env = StockTradingEnv(data) # Create the environment here

# Get the state size from the environment's observation space
state_size = env.observation_space.shape[0] # Get state_size here
action_size = env.action_space.n  # Get action_size here

# Create an instance of the DQNAgent
agent = DQNAgent(state_size, action_size) # Create the agent here


# Training parameters
episodes = 1000
batch_size = 32

for e in range(episodes):
    state = env.reset()
    state = np.reshape(state, [1, state_size])
    done = False
    total_reward = 0

    while not done:
        action = agent.act(state) # agent is now defined and can be used
        next_state, reward, done, _ = env.step(action)
        next_state = np.reshape(next_state, [1, state_size])
        agent.remember(state, action, reward, next_state, done)
        state = next_state
        total_reward += reward

        if len(agent.memory) > batch_size:
            agent.replay(batch_size)

    print(f"Episode: {e+1}/{episodes}, Total Reward: {total_reward}, Epsilon: {agent.epsilon:.2f}")

    if agent.epsilon > agent.epsilon_min:
        agent.epsilon *= agent.epsilon_decay

# Save the model
agent.save("dqn_trading_model.h5")

[*********************100%***********************]  1 of 1 completed
/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


ValueError: Argument(s) not recognized: {'lr': 0.001}